In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
data = pd.read_csv('C:/Bee Temp Data/three input.csv')
data.head()

In [ ]:
from lmfit import minimize, Parameters, Parameter, report_fit
import scipy.integrate as scint
import numpy as np
from matplotlib import pyplot as plt
import math
import warnings
import time
warnings.filterwarnings("ignore")
def bee_eq(y, t, w1,w2, external_temperature, treat, peri):
    hive_temperature = y
    u1 = [0]*len(hive_temperature)
    u2 = [0]*len(hive_temperature)
    for i in range(len(hive_temperature)):
        if external_temperature[i] >= 0:
            u1[i] = 1
        else:
            u2[i] = -1
    theta_adj = []
    for i in range(0,len(external_temperature),24):
        if treat == 0:
            theta_adj = np.concatenate((theta_adj, external_temperature[i:i+24]))
        else:
            theta_adj = np.concatenate((theta_adj, peri[i:i+24]))
    #print(theta_adj)
    dydt = -w1*hive_temperature*u1 + w2*hive_temperature*u2 + 2*(-hive_temperature) + external_temperature + theta_adj
    return dydt

def run_bee_eq(t, w1, w2, ext_temp, treat, peri):
    a = [10]*len(ext_temp)
    sol = scint.odeint(bee_eq, a, t, args=(w1,w2, ext_temp, treat, peri)) # 'w' and 'amplitude_temp_ext'
    theta_t = sol[-1,:]
    return theta_t

def residual(ps, ts, ext_temp, truth, treat, peri):
    #x0 = ps['a0'].value
    w1 = ps['w1'].value
    w2 = ps['w2'].value
    theta_ideal = ps['theta_ideal'].value
    ext_temp_ = ext_temp - theta_ideal
    peri_ = peri - theta_ideal
    truth_ = truth - theta_ideal
    model = run_bee_eq(ts, w1, w2, ext_temp_, treat, peri_)
    #print(w1,w2,theta_ideal)
    return (model - truth_).ravel()

In [ ]:
from scipy.stats import norm
import math
def segmentation(d):
    x = np.linspace(0, len(d['Temp'].values)-1, num = len(d['Temp'].values))
    t_max = len(d['Temp'].values)-1
    t = np.linspace(0,t_max, num = t_max+1)
    # set parameters incluing bounds
    params = Parameters()
    #params.add('a0', value=float(data['Temp_265'].values[0]), min=20, max=50)
    params.add('w1', value= 1.0, min=0, max=150)
    params.add('w2', value=1.0, min=0, max=150)
    params.add('theta_ideal', value = 35.0, min = min(d['Temp'].values), max = max(d['Temp'].values))
    result = minimize(residual, params, args=(t, d['Temp_Box'].values, d['Temp'].values,
                                              d['Treatment'].values[0], d['Temp_Easy'].values),
                      method='leastsq', nan_policy = 'propagate')
    pos = [0,len(d)]
    b = 1
    min_sig = []
    all_params = [[result.params['w1'].value, result.params['w2'].value, result.params['theta_ideal'].value]]
    all_positions = [0]
    all_lks = [np.sum(np.log(norm.pdf(d['Temp'].values.reshape(1,-1) -
                                      np.array(d['Temp'].values + result.residual.reshape(d['Temp'].values.shape)).reshape(1,-1))))]
    d_ = pd.DataFrame(d).groupby(data['Date'])
    while b < len(d_):
        position = []
        params_ = []
        signals = []
        likelihood = []
        for a in range(len(d_)):
            if a not in pos:
                positions = np.append(pos,a)
                positions.sort()
                sig_left = []
                left_params = []
                for i in range(len(positions)-1):
                    left_portion = d.iloc[positions[i]*24:positions[i+1]*24]
                    #print(left_portion)
                    x_left_grp = left_portion.groupby(left_portion['Date'])

                    for j,k in x_left_grp:
                        x_left = np.linspace(0, len(k['Temp'].values)-1, num = len(k['Temp']))
                        t_max = len(k['Temp'].values)-1
                        t = np.linspace(0,t_max, num = t_max+1)
                        xx_left =k['Temp_Box'].values
                        yy_left = k['Temp'].values
                        treat = k['Treatment'].values[0]
                        zz_left = k['Temp_Easy'].values
                        result_left = minimize(residual, params, args=(t, yy_left, xx_left, treat, zz_left) ,
                                               method='leastsq', nan_policy = 'propagate')
                        sig_left.append(xx_left + result_left.residual.reshape(xx_left.shape))
                    left_params.append([result_left.params['w1'].value,result_left.params['w2'].value,
                                       result_left.params['theta_ideal'].value])
                params_.append(left_params)
                position.append(positions)
                signals.append(sig_left)
                L = np.sum(np.log(norm.pdf(d['Temp'].values.reshape(1,-1) - np.array(sig_left).reshape(1,-1))))
                likelihood.append(-2*L + 2*((len(positions)-2)+(len(positions)-1)*3))
        predicted = signals[np.argmin(likelihood)]
        all_params.append(params_[np.argmin(likelihood)])
        #print(all_params)
        all_positions.append(position[np.argmin(likelihood)])
        all_lks.append(likelihood[np.argmin(likelihood)])
        min_sig.append(predicted)
        pos = position[np.argmin(likelihood)]
        min_lks = likelihood[np.argmin(likelihood)]
        b = b+1

    lower = np.argmin(all_lks)
    low_param = all_params[lower]
    print(low_param)
    return lower, low_param

In [ ]:
import numpy as np
for i in range(0,int(len(data)/24),10):
    d_ =  data[i*24:24*(i+10)]
    position, params = segmentation(d_[0:72])
    #print(position, params)
    if position == 0:
        data_in = []
        data_in_temp = d_[0:72]['Temp'].values
    else:
        data_in = d_[0:position*24]['Temp'].values
        data_in_temp = d_[position*24:72]['Temp'].values
    for j in range(1,8):
        data_in_ext = d_[position*24:len(data_in_temp)]['Temp_Box'].values
        data_in_peri = d_[position*24:len(data_in_temp)]['Temp_Easy'].values
        treat = d_[position*24:len(data_in_temp)]['Treatment'].values[0]
        data_for = d_[24*j:24*(j+1)]
        t_max = len(data_for['Temp_Box'])-1
        t = np.linspace(0,t_max, num = t_max+1)
        forecasted = run_bee_eq(t, params[0],params[1],
                                data_for['Temp_Box'].values - params[2],
                               treat, data_for['Temp_Easy'].values - params[2])
        data_in_temp = np.concatenate((data_in_temp, forecasted + params[2]))
        #print('hi')
    #plt.plot(data_in_temp)
    #plt.plot(d_['Temp'].values)
    #plt.show()
    data_in = np.concatenate((data_in, data_in_temp))
    for a in data_in:
        print(a)